In [1]:
import sys
sys.path.append("../../")
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.image as mpimg
import time
import pandas as pd
from pandas.plotting import scatter_matrix
import h5py
#from tqdm.keras import TqdmCallback
from transport_nets.models.NVP import NVP

tfd = tfp.distributions
tfb = tfp.bijectors

In [2]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
file = h5py.File('BeamSamples.h5','r')
joint_samples = np.array(file['Samples/'],dtype=np.float32)
scaler = MinMaxScaler()
scaled_samples = scaler.fit_transform(joint_samples)

NUM_SAMPLES = joint_samples.shape[0]
BATCH_SIZE = 50
dataset = tf.data.Dataset.from_tensor_slices(scaled_samples)
dataset = dataset.shuffle(buffer_size=NUM_SAMPLES).batch(BATCH_SIZE)

OUTPUT_DIM = 9
NUM_BLOCKS = 8
NEURON_LIST = [100,100]

masks = NUM_BLOCKS*[5]

nvp = NVP(output_dim=OUTPUT_DIM,
          num_layers = len(masks), 
          neuron_list = NEURON_LIST)

# Adam optimizer parameters
LR = 1e-4
beta_1 = 0.9
beta_2 = 0.999
opt = tf.keras.optimizers.Adam(learning_rate=LR, beta_1=beta_1, beta_2=beta_2, epsilon=1e-08)
nvp.compile(opt)

N_EPOCHS = 50
# Train the model with the new callback
nvp.batch_norm_mode(True)
history = nvp.fit(dataset, epochs=N_EPOCHS)  # Pass callback to training
# Set the training variables to not update when doing passes
nvp.batch_norm_mode(False)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 3, expecting 4
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 3, expecting 4
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
`AffineScalar` bijector is deprecated; please use `tfb.Shift(loc)(tfb.Scale(...))` instead.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
Please report this to the TensorFlow tea

TypeError: compile() got an unexpected keyword argument 'loss'

In [ ]:
from scipy.ndimage.filters import gaussian_filter1d
ysmoothed = gaussian_filter1d(history.history['Negative Log Likelihood'], sigma=2)
plt.plot(history.history['Negative Log Likelihood'],'b')
plt.plot(ysmoothed, color = 'r')
plt.xlabel('epochs')
plt.ylabel('negative log likelihood');

In [ ]:
plt.figure(figsize=(15,15))
scatter_matrix(pd.DataFrame(joint_samples[::100,:]))
plt.savefig('Euler-Beam-joint-density-samples.png')
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,15))
x_forward = scaler.inverse_transform(nvp.sample(500).numpy())
scatter_matrix(pd.DataFrame(x_forward))
plt.savefig('Euler-Beam-joint-density-nvp.png')
plt.show()

In [ ]:
from transport_nets.mcmc.metropolis_hastings import MetropolisHastings
from transport_nets.mcmc.metropolis_hastings import model_log_prob
num_params = 3
m_y_scaled = scaled_samples[0,:]
y_obs = m_y_scaled[num_params:]
init_val = 0.5*tf.ones([3],tf.float32)
b = tf.constant(0.05,tf.float32)
niters = 10000
log_prob_fn = model_log_prob(nvp,y_given=y_obs)

samples,acc_rate,time_total = MetropolisHastings(init_val,b,niters,log_prob_fn)

In [ ]:
# unscaling sampled parameters (m1,m2,m3)
data_min = scaler.data_min_[:y_s]
data_max = scaler.data_max_[:y_s]
scale = scaler.scale_[:y_s]
samples_unscaled = samples*(data_max-data_min) + data_min

fig, axs = plt.subplots(1,3,figsize=(15,5))
m_y_obs_unscaled = scaler.inverse_transform(np.reshape(m_y_scaled,[1,-1]))
m1,m2,m3 = tuple(m_y_obs_unscaled[0,:num_params])

axs[0].hist(samples_unscaled[:,0],50,density=True)
axs[0].axvline(m1,ymax=0.95,c='r',ls='--',lw=2.0,label='true val')
axs[0].set_xlabel('m1')
axs[0].legend()

axs[1].hist(samples_unscaled[:,1],50,density=True)
axs[1].axvline(m2,ymax=0.95,c='r',lw=2.0,ls='--',label='true val')
axs[1].set_xlabel('m2')
axs[1].legend()

axs[2].hist(samples_unscaled[:,2],50,density=True)
axs[2].axvline(m3,ymax=0.95,c='r',lw=2.0,ls='--',label='true val')
axs[2].set_xlabel('m3')
axs[2].legend()

plt.suptitle('Number of Samples: %d\n Acceptance Rate: %0.3f \n Time: %0.2f s'%(niters,acc_rate,time_total))
plt.subplots_adjust(top=0.85)

plt.savefig('Euler_Beam_MCMC.png')